# Morpheus analyses limit

## Table of content (ToC)<a class="anchor" id="TOC"></a>
* <a href="#bullet1">1 - Introduction</a>
* <a href="#bullet2">2 - Code exploration</a>
* <a href="#bullet3">3 - Testing the API</a>
    * <a href="#bullet3x1">3.1 - A plain API call</a>
    * <a href="#bullet3x2">3.2 - Using morphkit</a>
* <a href="#bullet4">4 - Conclusion</a>
* <a href="#bullet5">5 - Notebook version</a>

# 1 - Introduction <a class="anchor" id="bullet1"></a>
##### [Back to ToC](#TOC)

While running a large analyses run (when using [bash_script.sh](../Running_Morpheus_on_docker/bash_script.html)), the following was reported to `STDERR`:

```txt
diaperw=n:  ran out of space with 25 analyses!
diaperw=n:  ran out of space with 25 analyses!
diaperw=sin:  ran out of space with 25 analyses!
diaperw=sin:  ran out of space with 25 analyses!
diaperw=sin:  ran out of space with 25 analyses!
```

# 2 - Code exploration <a class="anchor" id="bullet2"></a>
##### [Back to ToC](#TOC)

The code generating this message is found in [checkgenwds.c](https://github.com/perseids-tools/morpheus/blob/ab6898ffed335fc6169fa02c9940657a9b5a78e0/src/anal/checkgenwds.c#L211):


```txt
    if( totanal_of(Gkword) >= MAXANALYSES ) {
        fprintf(stderr,"%s:  ran out of space with %d analyses!\n",
            rawword_of(Gkword), totanal_of(Gkword) );
        return(0);
```

This is set to 25 in the header file [gkstring.h](https://github.com/perseids-tools/morpheus/blob/ab6898ffed335fc6169fa02c9940657a9b5a78e0/src/includes/gkstring.h#L208):

```txt
#define MAXANALYSES 25
```


# 3 - Testing the API <a class="anchor" id="bullet3"></a>
##### [Back to ToC](#TOC)

Since we know the betacode for the 'offending' word, we can easily check the API responce.

## 3.1 - A plain API call <a class="anchor" id="bullet3x1"></a>
##### [Back to ToC](#TOC)

In [14]:
import requests
import re

# Define the API endpoint
api_endpoint = "http://10.0.1.156:1315/greek/diaperw=n?opts=n?opts=d"

# Make the API call
# Make the API call
def make_api_call(endpoint):
    try:
        response = requests.get(endpoint)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Make the API calls and store the results
result = make_api_call(api_endpoint)

# Display the results
if result:
    print("API Call Result:")
    print("\nHeaders:")
    for key, value in result.headers.items():
        print(f"{key}: {value}")
    print(f"\nThe string ':raw' is found {len(re.findall(':raw', result.text))} times.")
    print("\nResponse Content:")
    print(result.text)



API Call Result:

Headers:
Content-Type: text/html;charset=utf-8
X-XSS-Protection: 1; mode=block
X-Content-Type-Options: nosniff
X-Frame-Options: SAMEORIGIN
Content-Length: 4467

The string ':raw' is found 25 times.

Response Content:

:raw diaperw=n

:workw diaperw=n
:lem dia/,a)po/-e)ra/w1
:prvb dia/,a)po/				
:aug1 				
:stem e)r	 pres			aw_pr,a_stem
:suff 				
:end w=n	 pres part act masc voc sg		contr	aw_pr

:raw diaperw=n

:workw diaperw=n
:lem dia/,a)po/-e)ra/w1
:prvb dia/,a)po/				
:aug1 				
:stem e)r	 pres			aw_pr,a_stem
:suff 				
:end w=n	 pres part act neut nom/voc/acc sg		contr	aw_pr

:raw diaperw=n

:workw diaperw=n
:lem dia/,a)po/-e)ra/w1
:prvb dia/,a)po/				
:aug1 				
:stem e)r	 pres			aw_pr,a_stem
:suff 				
:end w=n	 pres part act masc nom sg	attic epic ionic	contr	aw_pr

:raw diaperw=n

:workw diaperw=n
:lem dia/,a)po/-e)ra/w2
:prvb dia/,a)po/				
:aug1 				
:stem e)r				aw_pr,a_stem
:suff 				
:end w=n	 pres part act masc voc sg		contr	aw_pr

:raw diaperw=n

:work

## 3.2 - Using Morphkit <a class="anchor" id="bullet3x2"></a>
##### [Back to ToC](#TOC)

Next see how [morphkit]() handles this case.

In [15]:
import sys
sys.path.insert(0, "../../morphkit")    # relative to notebook dir
import morphkit

In [16]:
base_url='http://10.0.1.156:1315/greek/'
dustbin=morphkit.get_word_blocks('diaperw=n', base_url, debug=True)

[get_word_blocks] GET http://10.0.1.156:1315/greek/diaperw%3Dn?opts=d?opts=n
[get_word_blocks] Status Code: 200
[get_word_blocks] Response Time: 0.040s
[get_word_blocks] Request Headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}
[get_word_blocks] Response Headers: {'Content-Type': 'text/html;charset=utf-8', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Length': '4467'}
[get_word_blocks] Response snippet (max 100 bytes):

:raw diaperw=n

:workw diaperw=n
:lem dia/,a)po/-e)ra/w1
:prvb dia/,a)po/				
:aug1 				
:stem e)r	...


# 4 - Conclusion <a class="anchor" id="bullet4"></a>
##### [Back to ToC](#TOC)

The STDERR output is not passed trough the API. Hence in case of using an API there seems to be no method to determine if when 25 blocks are recieved, the analyses is completer or not.

# 5 - Notebook version <a class="anchor" id="bullet5"></a>
##### [Back to ToC](#TOC)

<div style="float: left;">
  <table>
    <tr>
      <td><strong>Author</strong></td>
      <td>Tony Jurg</td>
    </tr>
    <tr>
      <td><strong>Version</strong></td>
      <td>1.0</td>
    </tr>
    <tr>
      <td><strong>Date</strong></td>
      <td>15 May 2025</td>
    </tr>
  </table>
</div>